---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1 

### (The auto-grader awarded 96/100 points for the code below.)

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [43]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head()

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
dtype: object

In [42]:
def date_sorter():
    
    import re
    import pandas as pd
    from datetime import datetime as dt
    
    # Clean up some typos in df
    cleaned_df = df.replace('Decemeber','December',regex=True).replace('Janaury','January',regex=True).replace('Age','Aug',regex=True)
    
    # Get the dates in the form of words
    format_one = cleaned_df.str.extract(r'((?:\d{,2}\s)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:-|\.|\s|,)\s?\d{,2}[a-z]*(?:-|,|\s)?\s?\d{2,4})')
    
    # Get the dates in the form of numbers
    format_two = cleaned_df.str.extract(r'((?:\d{1,2})(?:(?:\/|-)\d{1,2})(?:(?:\/|-)\d{2,4}))')
    
    # Get the dates where there is no days i.e only month and year  
    format_three = cleaned_df.str.extract(r'((?:\d{1,2}(?:-|\/))?\d{4})')
    
    # Merge 'format_one' with 'format_two' and 'format_three' 
    dates = format_one.fillna(format_two).fillna(format_three)
    
    dates = pd.Series(dates)
    
    possible_date_formats = ['%m/%d/%y','%d/%m/%y','%m/%d/%Y','%d/%m/%Y','%m-%d-%y','%d-%m-%y',
                             '%m-%d-%Y','%d-%m-%Y','%b %Y','%b. %Y','%b, %Y','%B %Y','%B, %Y','%d %b %Y',
                             '%d %B %Y','%d %b %y','%d %b. %Y','%d %b. %y','%d %B %y', '%B %d, %Y',
                             '%B %d %Y','%B. %d, %Y','%b %d, %Y','%b %d %Y','%b. %d, %Y','%b, %d, %Y','%m/%Y','%m-%Y','%Y',' %m/%d/%y',
                             ' %d/%m/%y',' %m/%d/%Y',' %d/%m/%Y',' %m-%d-%y',' %d-%m-%y',
                             ' %m-%d-%Y',' %d-%m-%Y',' %b %Y',' %b. %Y',' %b, %Y',' %B %Y',' %B, %Y',' %d %b %Y',
                             ' %d %B %Y',' %d %b %y',' %d %b. %Y',' %d %b. %y',' %d %B %y', ' %B %d, %Y',
                             ' %B %d %Y',' %B. %d, %Y',' %b %d, %Y',' %b %d %Y',' %b. %d, %Y',' %m/%Y',' %m-%Y',' %Y']
    
    
    '''possible_date_formats = [08/27/61, 27/08/61, 08/27/1961, 27/8/1961, 8-27-61, 27-8-61
                                8-27-1961, 27-8-1961, Aug 1961, Aug. 1961, Aug, 1961, August 1961, August, 1961, 27 Aug 1961, 
                                27 August 1961, 27 Aug 61, 27 Aug. 1961, 27 Aug, 61, 27 August 61, August 27, 1961, 
                                August 27 1961, August. 27, 1961, Aug 27, 1961, Aug. 27, 1961, Aug, 27, 1961, 8/1961, 8-1961, 1961]'''
    
    parsed_dates = []
    
    for raw_string_date in dates:
        for date_format in possible_date_formats:
            
    # Try to determine the date format of the raw string...once identifed, 
    # convert to the desired format of %m/%d/%y
            
            try:
                parsed_date = dt.strptime(raw_string_date, date_format).strftime('%Y/%m/%d') 
                
    # If correct format is found, don't test any other formats          
                
                break                                                         
        
    # If correct format is not found, keep trying other formats            
            
            except ValueError:    
                pass                                                        
        
        parsed_dates.append(parsed_date)
        
    final_df = pd.DataFrame()
    final_df['Old Index'] = dates.index
    final_df['New Date'] = parsed_dates
    final_df.sort_values('New Date',inplace=True)
    final_df.drop('New Date',axis=1,inplace=True)
    
    result = pd.Series(final_df['Old Index'].values)
    return result

date_sorter()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if sys.path[0] == '':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64